In [ ]:
import time
from tqdm import tqdm
import collections
import math
import pickle
from datetime import datetime
import numpy as np
import pandas as pd
import random
from gensim.models import Word2Vec

## 节省数据内存操作
- 读取用户点击日志、文章特征、文章文本emb

In [ ]:
# 节省数据内存
article_dtypes = {
    "article_id": "int32",
    "category_id": "int16",
    "created_at_ts": "int64",
    "words_count": "int16"}

click_log_dtypes = {
    "user_id": "int32",
    "click_article_id": "int32",
    "click_timestamp": "int64",
    "click_environment": "int8",
    "click_deviceGroup": "int8",
    "click_os": "int8",
    "click_country": "int8",
    "click_region": "int8",
    "click_referrer_type": "int8"}

def get_train_click_df(data_save_path, name='train_click_log.csv'):
    """获取训练样本"""
    train_df = pd.read_csv(data_save_path + name, dtype=click_log_dtypes)
    return train_df

def get_test_click_df(data_save_path, name='testA_click_log.csv'):
    """获取测试样本"""
    test_df = pd.read_csv(data_save_path + name, dtype=click_log_dtypes)
    return test_df

def get_item_info_df(data_save_path, name='articles.csv'):
    """获取文章特征"""
    item_info_df = pd.read_csv(data_save_path + name,  dtype=article_dtypes)
    item_info_df = item_info_df.rename(columns={'article_id': 'click_article_id'})
    return item_info_df

def get_item_emb_df(data_save_path, name='articles_emb.csv'):
    """获取文章文本emb"""
    emb_df = pd.read_csv(data_save_path + name)
    emb_df = emb_df.rename(columns={'article_id': 'click_article_id'})
    return emb_df

In [ ]:
data_path = "../data/"
offline = False # 离线测试，划分训练集与验证集
if offline:
    trn_df = get_train_click_df(data_path, "offline_trn_df.csv")
    val_df = get_test_click_df(data_path, "offline_val_df.csv")
else:
    trn_df = get_train_click_df(data_path, "train_click_log.csv")
    
item_info_df = get_item_info_df(data_path)
item_emb_df =  get_item_emb_df(data_path)

### 召回策略1：热度召回
- 普通方案：直接对新闻id做一个value_counts然后返回前5个文章（没有考虑文章的时效性）
- 我的方案：考虑每个用户最后一次点击的时间，返回一个区间内的热门文章，可设置为超参数

主要用于文本补充，当其他样本无法满足召回需求，则用热度召回补充

In [ ]:
"""
# 普通方案：获取近期点击最多的文章
def get_item_topk_list(df, k):
    topk_list = df['click_article_id'].value_counts().index[: k]
    return topk_list
"""

# 改进方案：获取近期点击最多的文章，区间[-1.8*10^8, 1*10^5]
def get_item_topk_click(click_df, k):
    click_df = click_df[['user_id', 'click_article_id', 'created_at_ts']]
    user_recall_items_dict = collections.defaultdict(dict)
    for user in tqdm(click_df['user_id'].unique()):
        user_last_click = user_item_time_dict[user][-1][1]
        target_df = click_df.loc[(click_df['created_at_ts'] < user_last_click + 1 * (10 ** 5)) & (click_df['created_at_ts'] > user_last_click - 1.8 * (10 ** 8))]
        
        tmp_df = target_df['click_article_id'].value_counts()[: k] / len(target_df['click_article_id'])
        user_recall_items_dict[user] = dict(tmp_df)
    return user_recall_items_dict

### 召回策略2：itemCF

i2i_sim物品相似度矩阵计算：
- **考虑文章的正向顺序点击和反向顺序点击**：如果是按照时间戳正向的话权重设为1，反向的话设一个小于1的超参数，因为考虑到网页间的图结构，经常是点完一个后下面有相关推荐，用户很可能点击下一个，反之则基本不会。
- **考虑文章的位置信息权重**：考虑到了随时间的信息衰减，如果两个文章点击的次序距离太远，做一个缩放，0.8**(np.abs(loc2 - loc1) – 1)，也设一个超参数。
- **考虑文章的点击时间权重**：这个是用户对物品的一个行为特征，与位置信息权重的想法类似，用一个指数来进行衰减。
- **考虑文章创建时间的点击权重**：物品本身的特征，与文章点击时间权重的处理方法相似。
- 其他的还考虑了物品之间Embedding向量之间的相似度、物品是否属于同一个类别（这个类别是官方提供的，有400多种，我不是很明白），是的话乘1.1权重，不是就乘1.0、还有文章的字数，这些我尝试过加到相似性计算中，效果不是很好，后来就没加。

itemCF推荐：
- **文章创建时间和用户点击时间差**：强特，每一个用户推荐时，找到最后一次点击时间戳，然后设一个最早可能点击时间和最晚可能点击时间，如果相似文章的创建时间不在这一个区间内，就直接pass。
- **文章创建时间差和相似文章对应文章在用户点击序列中的位置**：离最后一次点击越远的文章，其相似文章的权重就越低。

In [ ]:
def get_user_item_time(df, user_col='user_id', item_col='click_article_id', time_col='click_timestamp'):
    """
    根据点击时间获取用户的点击文章序列   
    return: dict, {user1: [(item1, time1), (item2, time2)..]...}
    """
    df = df[[user_col, item_col, time_col]]
    df = df.sort_values('click_timestamp')
    user_item_time_df = df.groupby(user_col)[item_col, time_col].apply(lambda group: make_item_time_tuple(group))\
                                                            .reset_index().rename(columns={0: 'item_time_list'})
    user_item_time_dict = dict(zip(user_item_time_df['user_id'], user_item_time_df['item_time_list']))
    
    return user_item_time_dict


def get_item_info_dict(df, item_col='click_article_id', time_col='created_at_ts', cate_col='category_id', word_cnt_col='words_count'):
    """
    获取文章id对应的基本属性 
    return: item_created_abs_time_dict, dict, {item: diff_timestamp}
            item_created_time_dict, dict, {item: created_timestamp}
            item_type_dict, dict, {item: cate}
            item_words_dict, dict, {item: word_cnt}
    """
    # 获取绝对时间字典
    item_created_abs_time_dict = dict(zip(df[item_col], df[time_col]))
    
    # 获取归一化时间字典
    max_min_scaler = lambda x : (x-np.min(x))/(np.max(x)-np.min(x))
    df[time_col] = df[[time_col]].apply(max_min_scaler)
    item_created_time_dict = dict(zip(df[item_col], df[time_col]))
    
    # 获取文章类型字典
    item_type_dict = dict(zip(df[item_col], df[cate_col]))
    
    # 获取文章字数字典
    item_words_dict = dict(zip(df[item_col], df[word_cnt_col]))
    
    return item_created_abs_time_dict, item_created_time_dict, item_type_dict, item_words_dict

def make_item_time_tuple(group_df, user_col='user_id', item_col='click_article_id', time_col='click_timestamp'):
    item_time_tuple = list(zip(group_df[item_col], group_df[time_col]))
    return item_time_tuple

In [ ]:
def itemcf_sim(df, item_created_time_dict, i2i_sim_save_path='./sim_matrix/'):
    """
    文章与文章之间的相似性矩阵计算
    return : i2i_sim, dict, {item1: {item2: weight2, item3: weight3, ...}}
    """
    user_item_time_dict = get_user_item_time(df)
    
    # 计算相似度
    i2i_sim = {}
    item_cnt = collections.defaultdict(int)
    for user, item_time_list in tqdm(user_item_time_dict.items()):
        for loc1, (i, i_click_time) in enumerate(item_time_list):
            item_cnt[i] += 1
            i2i_sim.setdefault(i, {})
            for loc2, (j, j_click_time) in enumerate(item_time_list):
                if i == j:
                    continue
                    
                # 考虑文章的正向顺序点击和反向顺序点击    
                loc_alpha = 1.0 if loc2 > loc1 else 0.7
                # 位置信息权重
                loc_weight = loc_alpha * (0.8 ** (np.abs(loc2 - loc1) - 1))
                # 点击时间权重
                click_time_weight = np.exp(0.8 ** np.abs(i_click_time - j_click_time))
                # 两篇文章创建时间的权重
                created_time_weight = np.exp(0.8 ** np.abs(item_created_time_dict[i] - item_created_time_dict[j]))
                
                i2i_sim[i].setdefault(j, 0)
                # 计算文章之间的相似度
                i2i_sim[i][j] += loc_weight * click_time_weight * created_time_weight / math.log(len(item_time_list) + 1)
    
    i2i_sim_ = i2i_sim.copy()
    # 两篇文章的流行度权重，惩罚过于热门的物品，此处popular_weight设置为0.4时效果较好
    popular_weight = 0.4
    for i, related_items in i2i_sim.items():
        for j, wij in related_items.items():
            tmpMax, tmpMin = max(item_cnt[i], item_cnt[j]), min(item_cnt[i], item_cnt[j])
            i2i_sim_[i][j] = wij / ((tmpMax ** popular_weight) * (tmpMin ** (1 - popular_weight)))
    
    # 将得到的相似性矩阵保存到本地
    pickle.dump(i2i_sim_, open(i2i_sim_save_path + 'itemcf_i2i_sim.pkl', 'wb'))
    
    return i2i_sim_

In [ ]:
def item_based_recommend(user_id, user_item_time_dict, i2i_sim, sim_item_topk, recall_item_num, topk_list, item_created_time_dict, item_created_abs_time_dict, emb_i2i_sim=None):
    """
    基于文章协同过滤的召回
    return: item_rank, list, [(item1, score1), (item2, score2)...]
    """
    # 获取用户历史交互的文章
    user_hist_items = user_item_time_dict[user_id]
    user_hist_items_ = {item_id for item_id, _ in user_hist_items}
    
    item_rank = {}
    for loc, (i, click_time) in enumerate(user_hist_items):
        for j, wij in sorted(i2i_sim[i].items(), key=lambda x: x[1], reverse=True)[: sim_item_topk]: # 先选取每篇文章前sim_item_topk个相似文章

            if j in user_hist_items_:
                # 用户已经看过j文章了
                continue
            
            # 文章生成及用户点击时间权重，设置一个区间，不在区间内的直接pass
            if item_created_abs_time_dict[j] > user_item_time_dict[user_id][-1][1] + 1 * (10 ** 5) or item_created_abs_time_dict[j] < user_item_time_dict[user_id][-1][1] - 1.8 * (10 ** 8):
                continue
                
            # 文章创建时间差权重
            created_time_weight = np.exp(0.9 ** np.abs(item_created_time_dict[i] - item_created_time_dict[j]))
            
            # 相似文章和历史点击文章序列中历史文章所在的位置权重
            loc_weight = (0.9 ** (len(user_hist_items) - loc))
            
            # embedding 相似性，效果不好，设置一个小权重，或者直接不用
            content_weight = 1.0
            if emb_i2i_sim:
                if emb_i2i_sim.get(i, {}).get(j, None) is not None:
                    content_weight += emb_i2i_sim[i][j]
                if emb_i2i_sim.get(j, {}).get(i, None) is not None:
                    content_weight += emb_i2i_sim[j][i]
                content_weight = np.exp(1.0005 ** content_weight)
            
            item_rank.setdefault(j, 0)
            item_rank[j] += content_weight * created_time_weight * loc_weight * wij
            
    # 不足10个，用热门商品补全
    if len(item_rank) < recall_item_num:
        for i, item in enumerate(topk_list):
            if item in item_rank.items(): # 填充的item应该不在原来的列表中
                continue
            item_rank[item] = - i - 100 # 随便给个负数就行
            if len(item_rank) == recall_item_num:
                break
    
    item_rank = sorted(item_rank.items(), key=lambda x: x[1], reverse=True)[: recall_item_num]
        
    return item_rank

### 召回策略3：swing

其实一开始我是想用UserCF也计算一下的，因为新闻本身的兴趣点就比较分散，用户更倾向于追求新闻的及时性和热点性，UserCF就能比较好地把握这一点。但实际跑起来的时候内存爆了，没办法用，但还是想把用户彼此之间的信息融入到我的召回模型里。所以找到了阿里的协同过滤推荐算法swing，
Swing的主要思想是基于图结构（u1,u2,i1），用户1和用户2都看过文章1和2，那说明1和2有关联，如果这种三元组数量越多，说明权重越大。
算法：计算两个用户阅读文章的交集，权重分子与之间item相似，分母则对两个用户的交集长度做了一个补充，如果这两个用户交集长度很小，则说明他们的兴趣差异很大，但同时看过这篇文章，说明这两篇文章相似度很高。

但还是爆内存了，这个图结构本质上是三阶特征，本身一二阶特征都够呛了，所以我考虑采样，方法是扔掉历史观看序列很长（大于8）的活跃用户样本，扔掉观看序列较小（小于3）的不活跃用户样本，剩下约63%的样本，然后进行计算。这个想法是我拍脑袋决定的，因为活跃用户和不活跃用户的参考价值都比较小，其实主要目的就是想至少算出来个数，来看看结果，结果还行，有0.18左右的得分。

In [ ]:
def swing(df, user_col='user_id', item_col='click_article_id', time_col='click_timestamp', swing_sim_save_path='./sim_matrix/'):
    """
    基于swing的召回
    return: sim_item_corr, dict, {item1: {item2: weight2, item3: weight3, ...}}
    """
    # item, (u1,t1), (u2, t2).....
    item_user_df = df.sort_values(by=[item_col, time_col])
    item_user_df = item_user_df.groupby(item_col).apply(lambda group: make_user_time_tuple(group, user_col, item_col, time_col)).reset_index().rename(
        columns={0: 'user_id_time_list'})
    
    item_user_time_dict = dict(zip(item_user_df[item_col], item_user_df['user_id_time_list']))
    
    # ((u1, u2), i1, d12)
    u_u_cnt = collections.defaultdict(list)
    item_cnt = collections.defaultdict(int)
    for item, user_time_list in tqdm(item_user_time_dict.items()):
        for u, u_time in user_time_list:
            item_cnt[item] += 1
            for relate_u, relate_u_time in user_time_list:
                if relate_u == u:
                    continue
                key = (u, relate_u) if u <= relate_u else (relate_u, u)
                u_u_cnt[key].append((item, np.abs(u_time - relate_u_time)))
                
    # (i1,i2), sim
    sim_item = {}
    alpha = 5.0
    for u_u, co_item_times in u_u_cnt.items():
        num_co_items = len(co_item_times)
        for i, i_time_diff in co_item_times:
            sim_item.setdefault(i, {})
            for j, j_time_diff in co_item_times:
                if j == i:
                    continue
                weight = 1.0  # np.exp(-15000*(i_time_diff + j_time_diff))
                sim_item[i][j] = sim_item[i].setdefault(j, 0.) + weight / (alpha + num_co_items)
                
    # norm by item count
    sim_item_corr = sim_item.copy()
    for i, related_items in sim_item.items():
        for j, cij in related_items.items():
            sim_item_corr[i][j] = cij / math.sqrt(item_cnt[i] * item_cnt[j])
    
    pickle.dump(sim_item_corr, open(swing_sim_save_path + 'swing_i2i_sim.pkl', 'wb'))
    return sim_item_corr

### 召回策略4：item2vec

将用户的点击序列扔到w2v模型里训练，得到每个新闻的Embedding向量，然后根据用户历史点击的新闻选取最相似的新闻推荐。（item2vec是没有时间窗概念的，认为序列中任意两个物品都相关），但比赛里我还是用的gensim库里的Word2Vec训练器（size: 表示词向量的维度。window：决定了目标词会与多远距离的上下文产生关系。）

In [ ]:
def item2vec(df, embed_size=64, emb_save_path='./embed/', split_char=' '):
    """
    基于word2vec的召回
    return: item_w2v_emb_dict, dict, {item1: embedding}
    """
    df = df.sort_values('click_timestamp')
    # 转换成字符串
    click_df['click_article_id'] = click_df['click_article_id'].astype(str)
    # 转换成句子的形式
    docs = click_df.groupby(['user_id'])['click_article_id'].apply(lambda x: list(x)).reset_index()
    docs = docs['click_article_id'].values.tolist()

    logging.basicConfig(format='%(asctime)s:%(levelname)s:%(message)s', level=logging.INFO)

    # window选择为5
    w2v = Word2Vec(docs, size=embed_size, sg=1, window=5, seed=2020, workers=24, min_count=1, iter=1)
    
    # 保存成字典的形式
    item_w2v_emb_dict = {k: w2v[k] for k in click_df['click_article_id']}
    pickle.dump(item_w2v_emb_dict, open(emb_save_path + 'item_w2v_emb.pkl', 'wb'))
    
    return item_w2v_emb_dict

### 离线测试

In [ ]:
# 获取所有数据的历史点击和最后一次点击，制作离线训练集和测试集
def get_hist_and_last_click(click_df):
    click_df = click_df.sort_values(by=['user_id', 'click_timestamp'])
    click_last_df = click_df.groupby('user_id').tail(1)
    
    # 如果用户只有一个点击，hist为空了，会导致训练的时候这个用户不可见，将数据放入到训练集
    def hist_func(user_df):
        if len(user_df) == 1:
            return user_df
        else:
            return user_df[: -1]
        
    click_hist_df = click_df.groupby('user_id').apply(hist_func).reset_index(drop=True)
    return click_hist_df, click_last_df

In [ ]:
# 依次评估召回的前10, 20, 30, 40, 50个文章中的击中率
def metrics_recall(user_recall_items_dict, trn_last_click_df, topk=5):
    last_click_item_dict = dict(zip(trn_last_click_df['user_id'], trn_last_click_df['click_article_id']))
    user_num = len(user_recall_items_dict)
    for k in range(10, 51, 10):
        hit_num = 0
        for user, item_list in user_recall_items_dict.items():
            # 获取前k个召回的结果
            tmp_recall_items = [x[0] for x in user_recall_items_dict[user][: k]]
            if last_click_item_dict[user] in set(tmp_recall_items):
                hit_num += 1
        
        hit_rate = round(hit_num * 1.0 / user_num, 5)
        print(' topk: ', k, ' : ', 'hit_num: ', hit_num, 'hit_rate: ', hit_rate, 'user_num : ', user_num)

In [ ]:
user_item_time_dict = get_user_item_time(trn_df)
item_created_abs_time_dict, item_created_time_dict, item_type_dict, item_words_dict = get_item_info_dict(item_info_df)
i2i_sim_save_path = "./sim_matrix/offline_itemcf_i2i_sim.pkl"
i2i_sim = itemcf_sim(total_df, item_created_time_dict, i2i_sim_save_path)
i2i_sim = pickle.load(open(i2i_sim_save_path, 'rb'))
topk_list = get_item_topk_list(total_df, 50)

In [ ]:
sim_item_topk = 100 # 选择与当前文章最相似的前sim_item_topk篇文章
recall_item_num = 50 # 最终召回sim_item_topk篇文章

import multiprocessing

user_recall_items_dict = multiprocessing.Manager().dict() # 多进程字典


for user in tqdm(trn_df['user_id'].unique()):
    user_recall_items_dict[user] = item_based_recommend(user, user_item_time_dict, \
                                                        i2i_sim, sim_item_topk, recall_item_num, \
                                                        topk_list, item_created_time_dict,item_created_abs_time_dict)

In [ ]:
submit_df = pd.read_csv("../data/sample_submit.csv")

In [ ]:
# 将字典的形式转换成df
user_item_score_list = []

for user, items in tqdm(user_recall_items_dict.items()):
    for item, score in items:
        user_item_score_list.append([user, item, score])

recall_df = pd.DataFrame(user_item_score_list, columns=['user_id', 'click_article_id', 'pred_score'])

In [ ]:
# 生成提交文件
def submit(recall_df, topk=5, model_name=None):
    recall_df = recall_df.sort_values(by=['user_id', 'pred_score'])
    recall_df['rank'] = recall_df.groupby(['user_id'])['pred_score'].rank(ascending=False, method='first')
    
    # 判断是不是每个用户都有5篇文章及以上
    tmp = recall_df.groupby('user_id').apply(lambda x: x['rank'].max())
    assert tmp.min() >= topk
    
    del recall_df['pred_score']
    submit = recall_df[recall_df['rank'] <= topk].set_index(['user_id', 'rank']).unstack(-1).reset_index()
    
    submit.columns = [int(col) if isinstance(col, int) else col for col in submit.columns.droplevel(0)]
    # 按照提交格式定义列名
    submit = submit.rename(columns={'': 'user_id', 1: 'article_1', 2: 'article_2', 
                                                  3: 'article_3', 4: 'article_4', 5: 'article_5'})
    
    save_name = './submit/' + datetime.today().strftime('%m-%d') + '.csv'
    submit.to_csv(save_name, index=False, header=True)

In [ ]:
submit(recall_df, topk=5, model_name=None)